# A quick start guide to using pjafroc

* `pjafroc` is the `python` implementation of `RJafroc`. However, not all `R` functions are implemented, only those essential to significance testing.
* Significance testing is the statistical analysis of ROC (or FROC) data to determine if there is a difference between the modalities or readers and, furthermore, quantifying the difference via relevant statistics, leading to the oft-quoted p-value and confidence intervals for estimates.
* It is assumed that the raw data consists of interpretations by observers of a set of cases containing diseased and non-diseased cases, in possibly multiple modalities.
* The following URLs, written in `Rmarkdown`, may be useful as they provide background material that is independent of the programming language: 
 * [The RJafroc Quick Start Book](https://dpc10ster.github.io/RJafrocQuickStart/)
 * [The RJafroc Roc Book](https://dpc10ster.github.io/RJafrocRocBook/)
 * [The RJafroc Froc Book](https://dpc10ster.github.io/RJafrocFrocBook/)

# Glossary of terms

* Treatment and modality are used interchangeably.
* Reader, radiologist or algorithmic observer are also used interchangeably.
* Rating: level of suspicion, recorded on an ordinal scale, with higher values associated with increasing confidence in presence of disease. 
* In the ROC paradigm each case interpretation results in **one** rating.
* In the FROC paradigm each case interpretation results in **zero or more** marked suspicious regions, each of which yields a rating reflecting confidence in presence of disease at each marked region. 
* FROC data consists of mark-rating pairs, where the number of mark-rating pairs on each case is a-priori unpredictable.
* K1 = number of non-diseased cases
* K2 = number of diseased cases
* K = K1 + K2 = total number of cases
* I = number of modalities
* J = number of readers

## Excel file format

* See [here](https://dpc10ster.github.io/RJafrocQuickStart/quick-start-data-format.html) for explanation of format of Excel file for ROC data and [here](https://dpc10ster.github.io/RJafrocQuickStart/quick-start-froc-data-format.html) for explanation of the format for FROC data. 
* NB: The final three columns in Excel file TRUTH worksheet, labeled ReaderID, ModalityID and Paradigm, can be omitted as they are ignored in the `pjafroc` implementation.
* Sample Excel data input files can be found in the `extdata` directory.
* The worksheet and column names are all case-sensitive.

In [1]:
import numpy as np

## Import necessary modules
* DfReadDataFile.py: read an Excel data file and return a dataset. 
* StSignificanceTesting.py: apply significance testing to an input dataset.
* UtilFigureOfMerit.py: compute figure of merit for each treatment-reader combination.
* UtilORVarComponents.py: compute variability components of dataset, used in significance testing.

In [2]:
from DfReadDataFile import DfReadDataFile, DfFroc2Roc, DfExtractDataset, DfRatings2Dataset
from StSignificanceTesting import StSignificanceTesting, StSignificanceTestingCadVsRad
from UtilFigureOfMerit import UtilFigureOfMerit
from UtilFigureOfMerit import UtilLesionWeightsDistr
from UtilORVarComponents import testJackKnife, UtilPseudoValues, UtilORVarComponents

## Read the Excel file

In [3]:
ds = DfReadDataFile("extdata/JT.xlsx", DataType="FROC")
len(ds)

7

* This reads the data file "extdata/JT.xlsx", specified as an FROC dataset, and returns a dataset object `ds`.  
* The `ds` object is a `list` containing 7 members: 
    * ds[0] = NL[0:I, 0:J, 0:K, 0:maxNL];
    * ds[1] = LL[0:I, 0:J, 0:K2, 0:maxLL];
    * ds[2] = `perCase`, array containing numbers of lesions per case
    * ds[3] = `relWeights`, the relative weights of lesions on diseased cases
    * ds[4] = `DataType`, the type of data, "FROC" or "ROC"
    * ds[5] = `modalityID`, names of modalities, strings
    * ds[6] = `readerID`, names of readers, strings

In [4]:
I = len(ds[0][:,0,0,0])
J = len(ds[0][0,:,0,0])
print("number of modalities = ", I, ", number of readers = ", J)

K = len(ds[0][0,0,:,0])
K2 = len(ds[1][0,0,:,0])
K1 = K - K2
print("number of non-diseased cases = ", K1, ", number of diseased cases = ", K2)

maxNL = len(ds[0][0,0,0,:])
maxLL = len(ds[1][0,0,0,:])
print("max number of NLs per case = ", maxNL, ", max number of LLs per diseased case = ", maxLL)

number of modalities =  2 , number of readers =  9
number of non-diseased cases =  45 , number of diseased cases =  47
max number of NLs per case =  7 , max number of LLs per diseased case =  3


In [5]:
ds[0][0,0,:10,:] # NL ratings for first treatment and first reader for the first 10 non-diseased cases
#ds[0][1,0,:10,:] # NL ratings for second treatment and first reader ...
# maximum number of NLs per case over entire dataset is 7

array([[  4.,   6., -inf, -inf, -inf, -inf, -inf],
       [  4., -inf, -inf, -inf, -inf, -inf, -inf],
       [  7., -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [  5.,   4., -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf]])

The dataset consists of K = 92 cases

In [6]:
K = len(ds[0][0,0,:,:]) # 92 cases in all
K

92

In [7]:
ds[1][0,0,:10,:] # LL ratings for first treatment and first reader and first 10 diseased cases
#ds[1][0,1,:,:] # LL ratings for first treatment and second reader
# maximum number of LLs per case over entire dataset is 3

array([[  5., -inf, -inf],
       [ 10., -inf, -inf],
       [  7., -inf, -inf],
       [  6.,   9., -inf],
       [-inf,   9., -inf],
       [-inf, -inf, -inf],
       [ 10., -inf, -inf],
       [ 10., -inf, -inf],
       [-inf, -inf, -inf],
       [  2., -inf, -inf]])

In [8]:
len(ds[0][0,0,:,:]) - len(ds[1][0,0,:,:]) # 45 non-diseased cases

45

**In FROC paradigm NLs can occur on non-diseased and diseased cases**

In [9]:
ds[0][0,0,45:55,:] # NL ratings for first treatment and first reader and first 10 diseased cases

array([[-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [  7.,   9., -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [  4., -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [  4., -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [  2., -inf, -inf, -inf, -inf, -inf, -inf]])

* The next statement displays the number of lesions per diseased case array, termed `perCase`
* The first diseased case has one lesion, the fourth has 2 lesions, etc.

In [10]:
ds[2]

array([1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 3, 2, 1, 1, 2, 1,
       1, 1, 1])

## Apply the significance testing procedure 

In [11]:
st = StSignificanceTesting(ds)

In [14]:
st[0]["foms"]

array([[0.72446809, 0.88096927, 0.80957447, 0.61331757, 0.57730496,
        0.84933018, 0.8927896 , 0.80260047, 0.76398739],
       [0.80236407, 0.96855792, 0.84598109, 0.75141844, 0.72092199,
        0.87186761, 0.93695823, 0.89948779, 0.81903073]])

In [15]:
st[0]["trtMeans"]

,Estimate
0,0.768260
1,0.846288


In [16]:
st[0]["trtMeanDiffs"]

,Estimate
trt0 - trt1,-0.078027


In [20]:
st[1]["TRAnova"]

,SS,DF,MS
T,0.027397,1,0.027397
R,0.146699,8,0.018337
TR,0.007422,8,0.000928


In [21]:
st[1]["VarCom"]

,Estimates,rhos
VarR,0.008427,
VarTR,0.000115,
Var,0.001662,
Cov1,0.000485,0.292114
Cov2,0.000571,0.343705
Cov3,0.000208,0.124859


In [22]:
st[1]["IndividualTrt"]

,DF,msREachTrt,varEachTrt,cov2EachTrt
trt0,8,0.012510,0.001920,0.000784
trt1,8,0.006756,0.001404,0.000358


In [23]:
st[1]["IndividualRdr"]

,DF,msTEachRdr,varEachRdr,cov1EachRdr
rdr0,1,0.003034,0.002276,0.000799
rdr1,1,0.003836,0.000691,0.000116
rdr2,1,0.000663,0.001584,0.000554
rdr3,1,0.009536,0.002243,0.000890
rdr4,1,0.010313,0.002637,0.001004
rdr5,1,0.000254,0.001416,0.000121
rdr6,1,0.000975,0.000745,0.000006
rdr7,1,0.004694,0.001321,0.000396
rdr8,1,0.001515,0.002046,0.000483


In [25]:
st[2]["FTests"]

[                 DF      MS   FStat  PValue
 Treatment    1.0000  0.0274  6.5212  0.0116
 Error      164.0528  0.0042     NaN     NaN]

In [26]:
st[2]["ciDiffTrt"]

[             Estimate  StdErr        DF       t   PrGTt   CI_lo   CI_hi
 trt0 - trt1    -0.078  0.0306  164.0528 -2.5537  0.0116 -0.1384 -0.0177]

In [27]:
st[2]["ciAvgRdrEachTrt"]

,Estimate,StdErr,DF,CI_lo,CI_hi,Cov2
trt0,0.768260,0.046626,19.570419,0.670863,0.865657,0.000784
trt1,0.846288,0.033303,17.465105,0.776167,0.916408,0.000358
